## Notebook for the Epithelial Reference Map and Joanito Cancer Epithelial cells preparation for labels transfer

- **Developed by**: Anna Maguza
- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**
- 31st May 2023

### Import required moduls

In [1]:
import scanpy as sc
import numpy as np
import anndata as ad
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

#### Setup Cells

In [2]:
%matplotlib inline

In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi = 160, color_map = 'magma_r', dpi_save = 300, vector_friendly = True)

/Users/anna.maguza/miniconda3/envs/scanpy_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.2.2 statsmodels==0.13.5 pynndescent==0.5.8


### Upload data

In [15]:
input = '/Users/anna.maguza/Desktop/Data/Processed_datasets/Cancer_dataset_integration/input_files/Epithelial_cells/Geosketch_subset/Epithelial_cells_Geosketch_subset_all_genes.h5ad'
Healthy_adata = sc.read(input)

In [16]:
input_cancer = '/Users/anna.maguza/Desktop/Data/Gut_project/Joanito_cancer/anndata/Joanito_raw_anndata_tumor_cells.h5ad'
Cancer_adata = sc.read(input_cancer)

### Find HVGs

In [17]:
Healthy_adata.raw.X

<31811x23616 sparse matrix of type '<class 'numpy.float32'>'
	with 68589169 stored elements in Compressed Sparse Row format>

In [18]:
Cancer_adata.raw.X

<170596x33287 sparse matrix of type '<class 'numpy.float32'>'
	with 304519512 stored elements in Compressed Sparse Row format>

In [19]:
Healthy_adata.obs['Library_Preparation_Protocol'].value_counts()    

10x 3' v1    11022
10x 5' v1    10753
10x 3' v2     6589
10x 3' v3     3447
Name: Library_Preparation_Protocol, dtype: int64

In [20]:
Healthy_adata.layers['raw_counts'] = Healthy_adata.X.copy()

### HVGs selection
# Calculate HVGs for cancer dataset
sc.pp.highly_variable_genes(
    Healthy_adata,
    flavor = "seurat_v3",
    n_top_genes = 2000,
    layer = "raw_counts",
    batch_key = "Library_Preparation_Protocol",
    subset = True,
    span = 1
)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


In [21]:
# Filter epithelial cells
Cancer_adata = Cancer_adata[Cancer_adata.obs['Cell Type'] == 'Epithelial',:]

In [22]:
Cancer_adata.layers['raw_counts'] = Cancer_adata.X.copy()

In [23]:
# Extract same HVGs in the cancer dataset as in the healthy dataset

#Make indexes as string
Cancer_adata.var.index = Cancer_adata.var.index.astype(str)

# Ensure indexes are unique
Cancer_adata.var_names_make_unique()

# Identify common genes
common_genes = list(set(Healthy_adata.var_names) & set(Cancer_adata.var_names))

# Filter genes
Healthy_adata = Healthy_adata[:, common_genes]
Cancer_adata = Cancer_adata[:, common_genes]

#Ensure the same order of the genes
Cancer_adata = Cancer_adata[:, Healthy_adata.var_names]

In [24]:
# Save anndata objects
Healthy_adata.write('/Users/anna.maguza/Desktop/Data/Processed_datasets/Cancer_dataset_integration/input_files/Epithelial_cells/Geosketch_subset/Joanito/Healthy_epithelial_cells_Geosketch_subset_2000_HVGs.h5ad')
Cancer_adata.write('/Users/anna.maguza/Desktop/Data/Processed_datasets/Cancer_dataset_integration/input_files/Epithelial_cells/Geosketch_subset/Joanito/Joanito_cancer_epithelial_cells_2000_HVGs.h5ad')